Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Abstractive Summarization on CNN/DM Dataset using Transformers


### Summary

This notebook demonstrates how to fine tune Transformers for extractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.




### Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Ubuntu Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

Using only 1 NVIDIA Tesla V100 GPUs, 16GB GPU memory configuration,
- for data preprocessing, it takes around 1 minutes to preprocess the data for quick run. Otherwise it takes ~20 minutes to finish the data preprocessing. This time estimation assumes that the chosen transformer model is "distilbert-base-uncased" and the sentence selection method is "greedy", which is the default. The preprocessing time can be significantly longer if the sentence selection method is "combination", which can achieve better model performance.

- for model fine tuning, it takes around 2 minutes for quick run. Otherwise, it takes around ~3 hours to finish. This estimation assumes the chosen encoder method is "transformer". The model fine tuning time can be shorter if other encoder method is chosen, which may result in worse model performance. 

### Additional Notes

* **ROUGE Evalation**: To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb) for setup.

* **Distributed Training**:
Please note that the jupyter notebook only allows to use pytorch [DataParallel](https://pytorch.org/docs/master/nn.html#dataparallel). Faster speed and larger batch size can be achieved with pytorch [DistributedDataParallel](https://pytorch.org/docs/master/notes/ddp.html)(DDP). Script [extractive_summarization_cnndm_distributed_train.py](./extractive_summarization_cnndm_distributed_train.py) shows an example of how to use DDP.



In [1]:
%load_ext autoreload

%autoreload 2
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = False


### Configuration


In [2]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset
from utils_nlp.eval import compute_rouge_python, compute_rouge_perl
from utils_nlp.models.transformers.abstractive_summarization_bartt5 import AbstractiveSummarizer, SummarizationProcessor

from utils_nlp.models.transformers.datasets import SummarizationDataset
import nltk
from nltk import tokenize

import pandas as pd
import scrapbook as sb
import pprint

/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu


### Configuration: choose the transformer model to be used

Several pretrained models have been made available by [Hugging Face](https://github.com/huggingface/transformers). For extractive summarization, the following pretrained models are supported. 

In [3]:
#pd.DataFrame({"model_name": ExtractiveSummarizer.list_supported_models()})

In [3]:
# Transformer model being used
#MODEL_NAME = "t5-large"
MODEL_NAME = "bart-large-cnn"

In [4]:
# notebook parameters
# the cache data path during find tuning
CACHE_DIR = TemporaryDirectory().name
        
processor = SummarizationProcessor(MODEL_NAME,cache_dir=CACHE_DIR ) #tokenizer, config.prefix)

### Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples.   The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.


In [5]:
# the data path used to save the downloaded data file
DATA_PATH = "./bartt5_cnndm" #TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 1000
if not QUICK_RUN:
    TOP_N = -1

In [6]:
train_dataset, test_dataset = CNNDMSummarizationDataset(top_n=TOP_N, local_cache_path=DATA_PATH, raw=True)

In [7]:
len(test_dataset)

11490

Preprocess the data.

In [ ]:
%time
abs_sum_train = processor.preprocess(train_dataset)


CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 9.78 µs


In [1]:
abs_sum_train[0].keys()

NameError: name 'abs_sum_train' is not defined

In [18]:
abs_sum_train

NameError: name 'abs_sum_train' is not defined

In [10]:
abs_sum_test = processor.preprocess(test_dataset)

In [11]:
abs_sum_test[0]

{'src': "marseille , france -lrb- cnn -rrb- the french prosecutor leading an investigation into the crash of germanwings flight 9525 insisted wednesday that he was not aware of any video footage from on board the plane . marseille prosecutor brice robin told cnn that `` so far no videos were used in the crash investigation . '' he added , `` a person who has such a video needs to immediately give it to the investigators . '' robin 's comments follow claims by two magazines , german daily bild and french paris match , of a cell phone video showing the harrowing final seconds from on board germanwings flight 9525 as it crashed into the french alps . all 150 on board were killed . paris match and bild reported that the video was recovered from a phone at the wreckage site . the two publications described the supposed video , but did not post it on their websites . the publications said that they watched the video , which was found by a source close to the investigation . `` one can hear c

In [8]:
"""
# save and load preprocessed data
save_path = DATA_PATH
torch.save(abs_sum_train, os.path.join(save_path, "train_full.pt"))
torch.save(abs_sum_test, os.path.join(DATA_PATH, "test_full.pt"))

"""
save_path = DATA_PATH
#abs_sum_train = torch.load(os.path.join(save_path, "train_full.pt"))
abs_sum_test = torch.load(os.path.join(save_path, "test_full.pt"))

In [9]:
print(len(abs_sum_train))
print(len(abs_sum_test))

287227
11490


In [13]:
#save_path = os.path.join(DATA_PATH, "processed")
#torch.save(abs_sum_train, os.path.join(save_path, "train_full.pt"))
#torch.save(abs_sum_test, os.path.join(DATA_PATH, "test_full.pt"))

#### Inspect Data

In [ ]:
abs_sum_train[0].keys()

In [ ]:
abs_sum_train[0]

### Model training
To start model training, we need to create a instance of ExtractiveSummarizer.
#### Choose the transformer model.
Currently ExtractiveSummarizer support two models:
- distilbert-base-uncase, 
- bert-base-uncase

Potentionally, roberta-based model and xlnet can be supported but needs to be tested.
#### Choose the encoder algorithm.
There are four options:
- baseline: it used a smaller transformer model to replace the bert model and with transformer summarization layer
- classifier: it uses pretrained BERT and fine-tune BERT with **simple logistic classification** summarization layer
- transformer: it uses pretrained BERT and fine-tune BERT with **transformer** summarization layer
- RNN: it uses pretrained BERT and fine-tune BERT with **LSTM** summarization layer

In [9]:
BATCH_SIZE = 5 # batch size, unit is the number of samples
MAX_POS_LENGTH = 512

    


# GPU used for training
NUM_GPUS = torch.cuda.device_count()


# Learning rate
LEARNING_RATE=2e-3

# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=100

# total number of steps for training
MAX_STEPS=1e2
# number of steps for warm up
WARMUP_STEPS=5e2
    
if not QUICK_RUN:
    MAX_STEPS=5e4
    WARMUP_STEPS=5e3
 

In [10]:
summarizer = AbstractiveSummarizer(processor, MODEL_NAME)

In [ ]:
"""

summarizer.fit(
            ext_sum_train,
            num_gpus=NUM_GPUS,
            batch_size=BATCH_SIZE,
            gradient_accumulation_steps=2,
            max_steps=MAX_STEPS,
            learning_rate=LEARNING_RATE,
            warmup_steps=WARMUP_STEPS,
            verbose=True,
            report_every=REPORT_EVERY,
            clip_grad_norm=False,
            use_preprocessed_data=USE_PREPROCSSED_DATA
        )

"""


In [ ]:
"""
summarizer.save_model(
    os.path.join(
        CACHE_DIR,
        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(
            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS
        ),
    )
)
"""


In [ ]:
# for loading a previous saved model
"""
import torch
model_path = os.path.join(
        CACHE_DIR,
        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(
            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS
        ))
summarizer = ExtractiveSummarizer(processor, MODEL_NAME, ENCODER, MAX_POS_LENGTH, CACHE_DIR)
summarizer.model.load_state_dict(torch.load(model_path, map_location="cpu"))
"""

### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluating text summarization.

In [ ]:
abs_sum_test[0].keys()

In [8]:
source = []
target = []
for i in abs_sum_test:
    source.append(i["src_txt"]) 
    target.append(i['tgt'].replace("<t>","").replace("</t>", "").replace("\n", "")) 

In [13]:
target[0]

" marseille prosecutor says `` so far no videos were used in the crash investigation '' despite media reports .   journalists at bild and paris match are `` very confident '' the video clip is real , an editor says .   andreas lubitz had informed his lufthansa training school of an episode of severe depression , airline says . "

In [15]:
%%time
prediction = summarizer.predict(abs_sum_test[0:256*4], num_gpus=1, batch_size=16) 

Generating summary:   0%|          | 0/64 [00:00<?, ?it/s]

dataset length is 1024


Generating summary: 100%|██████████| 64/64 [07:29<00:00,  6.67s/it]


CPU times: user 7min 25s, sys: 3.61 s, total: 7min 29s
Wall time: 7min 30s


In [16]:

%%time
prediction = summarizer.predict(abs_sum_test[0:256*4], num_gpus=2, batch_size=32) 

Generating summary:   0%|          | 0/32 [00:00<?, ?it/s]

dataset length is 1024


Generating summary: 100%|██████████| 32/32 [06:01<00:00, 10.43s/it]


CPU times: user 8min 21s, sys: 52.5 s, total: 9min 13s
Wall time: 6min 2s


In [17]:

%%time
prediction = summarizer.predict(abs_sum_test[0:256*4], num_gpus=NUM_GPUS, batch_size=64) 

Generating summary:   0%|          | 0/16 [00:00<?, ?it/s]

dataset length is 1024


Generating summary: 100%|██████████| 16/16 [04:59<00:00, 17.81s/it]


CPU times: user 9min 11s, sys: 1min 25s, total: 10min 37s
Wall time: 5min


In [ ]:
torch.save(prediction, "prediction.pt")

In [3]:
prediction = torch.load("prediction.pt")

In [10]:
rouge_scores = compute_rouge_python(cand=prediction, ref=target)
pprint.pprint(rouge_scores)

Number of candidates: 11490
Number of references: 11490
{'rouge-1': {'f': 0.43366650568906373,
             'p': 0.3878160218865652,
             'r': 0.5256684651435454},
 'rouge-2': {'f': 0.2013283037622797,
             'p': 0.18073246915601657,
             'r': 0.24341426947272182},
 'rouge-l': {'f': 0.2945220878967815,
             'p': 0.26331547921506626,
             'r': 0.3573184457546521}}


In [ ]:
target[0]

In [ ]:
prediction[0]

In [ ]:
source[0]

In [ ]:
# for testing
sb.glue("rouge_2_f_score", rouge_scores['rouge-2']['f'])

## Prediction on a single input sample

In [ ]:
source = """
But under the new rule, set to be announced in the next 48 hours, Border Patrol agents would immediately return anyone to Mexico — without any detainment and without any due process — who attempts to cross the southwestern border between the legal ports of entry. The person would not be held for any length of time in an American facility.

Although they advised that details could change before the announcement, administration officials said the measure was needed to avert what they fear could be a systemwide outbreak of the coronavirus inside detention facilities along the border. Such an outbreak could spread quickly through the immigrant population and could infect large numbers of Border Patrol agents, leaving the southwestern border defenses weakened, the officials argued.
The Trump administration plans to immediately turn back all asylum seekers and other foreigners attempting to enter the United States from Mexico illegally, saying the nation cannot risk allowing the coronavirus to spread through detention facilities and Border Patrol agents, four administration officials said.
The administration officials said the ports of entry would remain open to American citizens, green-card holders and foreigners with proper documentation. Some foreigners would be blocked, including Europeans currently subject to earlier travel restrictions imposed by the administration. The points of entry will also be open to commercial traffic."""

In [ ]:
test_dataset = SummarizationDataset(
    None,
    source=[source],
    source_preprocessing=[tokenize.sent_tokenize],
    word_tokenize=nltk.word_tokenize,
)
processor = ExtSumProcessor(model_name=MODEL_NAME,  cache_dir=CACHE_DIR)
preprocessed_dataset = processor.preprocess(test_dataset)

In [ ]:
preprocessed_dataset[0].keys()

In [ ]:
prediction = summarizer.predict(preprocessed_dataset, num_gpus=0, batch_size=1, sentence_separator="\n")

In [ ]:
prediction

## Clean up temporary folders

In [ ]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR, ignore_errors=True)
if USE_PREPROCSSED_DATA:
    if os.path.exists(PROCESSED_DATA_PATH):
        shutil.rmtree(PROCESSED_DATA_PATH, ignore_errors=True)